In [1]:
import numpy as np
import pandas as pd

import random

from sklearn.datasets import make_regression

In [2]:
X, y = make_regression(n_samples=1000, n_features=14, n_informative=10, noise=15, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]

In [3]:
display(X.head())
print(X.shape)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13
0,1.191261,-0.308960,0.675137,-1.460501,0.053059,-1.886129,2.710794,-1.716033,0.865290,0.138078,-0.063745,-2.104583,-0.476876,1.677116
1,1.071476,-1.424766,-1.109750,-0.457677,0.399997,1.587401,-1.547570,0.323247,0.165859,-0.302097,0.203944,-0.212452,0.836991,0.368498
2,-0.045929,1.868603,-0.016568,-0.484258,1.089905,-1.147160,0.590744,0.683325,-0.571184,-0.802199,-0.220114,0.034808,0.043829,0.955803
3,-1.487154,2.220322,0.718332,1.682888,-0.420986,-0.054746,1.900832,-0.101198,0.090042,-0.202924,0.340865,0.606237,-0.037008,-0.841048
4,0.344054,0.657763,0.348342,-0.417430,-0.589112,1.057814,-0.487705,-0.897830,-0.935596,-1.186993,1.074333,-0.069532,-0.177918,-0.912811


(1000, 14)


In [4]:
display(y.head())
print(y.shape)

0    -48.005272
1    145.801614
2    -49.114775
3     24.902238
4   -152.611643
dtype: float64

(1000,)


In [5]:
class MyLineReg():

    def __init__(self, n_iter = 100, learning_rate = 0.1, metric = None, reg = None, l1_coef = 0, l2_coef = 0, sgd_sample = None, random_state = 42):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        random.seed(random_state)
        self.weights = None


    def __repr__(self):
        return f'MyLineReg: n_iter={self.n_iter}, learning_rate={self.learning_rate}'


    def fit(self, X: pd.DataFrame, y: pd.Series, verbose: int = False):
        X = X.copy()
        X.reset_index(inplace=True, drop=True)
        X.insert(loc=0, column='x0', value=1)
        self.weights = np.ones(X.shape[1])

        for iter in range(1, self.n_iter+1):

            if self.sgd_sample:
                if 0<self.sgd_sample<1:
                    sgd = round(self.sgd_sample * X.shape[0])
                else:
                    sgd = self.sgd_sample
                sample_rows_idx = random.sample(range(X.shape[0]), sgd)
                X_batch, y_batch = X.iloc[sample_rows_idx], y.iloc[sample_rows_idx]
            else:
                X_batch, y_batch = X, y

            pred = np.dot(X_batch, self.weights)
            grad = 2*(np.dot((pred-y_batch), X_batch)/len(y_batch))

            if self.reg:
                if self.reg == 'l1':
                    grad += self.l1_coef * np.sign(self.weights)
                if self.reg == 'l2':
                    grad += self.l2_coef * 2 * self.weights
                if self.reg == 'elasticnet':
                    grad += self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights

            if callable(self.learning_rate):
                lr = self.learning_rate(iter)
            else:
                lr = self.learning_rate

            self.weights -= grad*lr

            if verbose:

                if self.reg:
                    if self.reg == 'l1':
                        reg = self.l1_coef * np.sum(np.abs(self.weights))
                    if self.reg == 'l2':
                        reg = self.l2_coef * np.sum((self.weights)**2)
                    if self.reg == 'elasticnet':
                        reg = self.l1_coef * np.sum(np.abs(self.weights))
                        reg += self.l2_coef * np.sum((self.weights)**2)
                else:
                    reg = 0

                if iter==1:
                    pred = np.dot(X, np.ones(X.shape[1]))
                    loss = np.mean((pred-y)**2) + reg
                    res = f'start | loss: {loss}'
                    print_flag=True

                if iter%verbose==0:
                    pred = np.dot(X, self.weights)
                    loss = np.mean((pred-y)**2) + reg
                    res = f'{iter} | loss: {loss}'
                    print_flag=True
                    
                if self.metric:
                    pred = np.dot(X, self.weights)
                    metric_val = self.metric_calc(pred, y)
                    res += f' | {self.metric}: {metric_val}'
                    
                if callable(self.learning_rate):
                    res += f' | lr: {lr}'

                if print_flag:
                    print(res)
                    print_flag=False

    
    def metric_calc(self, pred: pd.Series, y: pd.Series):
        if self.metric:
            if self.metric == 'mae':
                return np.mean(np.abs(y - pred))
            if self.metric == 'mse':
                return np.mean((y-pred)**2)
            if self.metric == 'rmse':
                return np.mean((y-pred)**2)**(1/2)
            if self.metric == 'mape':
                return 100*np.mean(np.abs((y-pred)/y))
            if self.metric == 'r2':
                return 1 - (np.sum((y-pred)**2)/(np.sum((y-np.mean(y))**2)))


    def predict(self, X: pd.DataFrame):
        X = X.copy()
        X.insert(loc=0, column='x0', value=1)
        pred = np.dot(X, self.weights)
        return pred


    def get_coef(self):
        return self.weights[1:]
    

    def get_best_score(self):
        return self.metric_calc(self.predict(X), y)

In [6]:
mlg = MyLineReg(metric='rmse', sgd_sample=0.1)
print(mlg)

MyLineReg: n_iter=100, learning_rate=0.1


In [7]:
mlg.fit(X, y, verbose=10)

start | loss: 20621.089638778492 | rmse: 121.27867217648222
10 | loss: 513.0795282239593 | rmse: 22.65125886620784
20 | loss: 229.76421726629312 | rmse: 15.157975368310014
30 | loss: 226.15290691614268 | rmse: 15.038381126841502
40 | loss: 227.0865849200188 | rmse: 15.069392320860812
50 | loss: 227.15154947470484 | rmse: 15.071547680139052
60 | loss: 228.22525872980785 | rmse: 15.107126091014393
70 | loss: 227.0407967766683 | rmse: 15.067873001079757
80 | loss: 225.42368069407732 | rmse: 15.014116047709146
90 | loss: 226.2193427922896 | rmse: 15.040589841900802
100 | loss: 227.49742794016169 | rmse: 15.083017865803969


In [8]:
mlg.get_coef()

array([43.82645862, 16.20706114,  0.28237269, 64.97095574, 47.64099559,
       61.98602636,  0.2893713 , -0.8631572 , 59.83902748, 52.94121571,
        0.10169989, 14.98938684, 18.15834046, 47.5221333 ])

In [9]:
mlg.predict(X)[:5]

array([ -60.43910878,  134.6256443 ,  -52.12672252,   21.26701247,
       -128.91112088])

In [10]:
mlg.get_best_score()

15.083017865803969